In [ ]:
import numpy as np
import tensorflow as tf
import keras.datasets as ds  # keras 별도 라이브러리

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam

In [ ]:
# 1) 데이터 준비
(x_train,y_train),(x_test,y_test)=ds.mnist.load_data()
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

x_train=x_train.reshape(60000,784)
x_test=x_test.reshape(10000,784)

# 입력값(0~255)을 0~1로 정규화
x_train=x_train.astype(np.float32)/255.0
x_test=x_test.astype(np.float32)/255.0
# to_categorical : 정수 형식의 클래스 레이블을 one-hot 형식으로 변환하는 데 사용
y_train=tf.keras.utils.to_categorical(y_train,10)
y_test=tf.keras.utils.to_categorical(y_test,10)

In [ ]:
# K-fold 준비
from sklearn.model_selection import KFold

# train & test 모두 결합
inputs = np.concatenate((x_train, x_test), axis=0)
outputs = np.concatenate((y_train, y_test), axis=0)

# K-fold Cross Validator 정의
num_folds = 5
kfold = KFold(n_splits=num_folds, shuffle=True)

In [ ]:
# 각 K-fold Cross Validation model 학습 & 검증 실행
fold_no = 1
for train, test in kfold.split(inputs, outputs):

  # 2) 모델
  model=Sequential()
  model.add(Dense(units=1024,activation='relu',input_shape=(784,)))
  model.add(Dense(units=512,activation='relu'))
  model.add(Dense(units=512,activation='relu'))
  model.add(Dense(units=10,activation='softmax'))

  # 3) 학습
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.0001),metrics=['accuracy'])
  model.fit(inputs[train], outputs[train],batch_size=128,epochs=50,verbose=2)

  # 4) 예측
  scores = model.evaluate(inputs[test], outputs[test], verbose=0)
  print('정확률 for fold', {fold_no}, ' = ', scores[1]*100)

  # Increase fold number
  fold_no = fold_no + 1